In [ ]:
import numpy as np
import pandas as pd
import re
import pytrec_eval
import json

In [ ]:
def min_edit_dist(t1,t2):
    m,n = len(t1)+1,len(t2)+1
    distance_mat = np.zeros([m,n])
    for j in range(n):
        distance_mat[0][j] = j
    for i in range(m):
        distance_mat[i][0] = i
    distance_mat
    cost=0
    
    for i in range(1,m):
        for j in range(1,n):
            if(t1[i-1] == t2[j-1]):
                cost=0
            else:
                cost=1
            a = distance_mat[i-1][j]+1
            b = distance_mat[i,j-1]+1
            c = distance_mat[i-1,j-1]+cost
            distance_mat[i,j]=min(a,b,c)
    return distance_mat[m-1,n-1]

In [ ]:
text,c = [],[]
with open("courps.txt") as r:
    for t in r.readlines():
        if(re.match(r"\A[$]",t)):
            c.append(t)
        else:
            text.append(t)

In [ ]:
misspelled = []
spelled = []
t3 = []
count = 0
for t in text:
    t1 = re.split(r"\s+",t)
    for t2 in t1:
        if(re.match(r".\w",t2)):
            if((re.sub(r"[^a-zA-Z]","",t2)).isalpha()):
                t3.append(re.sub(r"[^a-zA-Z]","",t2))
            
for t in t3:
    count=count+1
    if(count%2 == 1):
        misspelled.append(t)
    else:
        spelled.append(t)

In [ ]:
from nltk.corpus import words as wn

In [ ]:
def sim_word(wordlist, miss_word):
    l = len(wordlist)
    p = min_edit_dist(miss_word,wordlist[0])
    dis = []
    for i in range(1,l):
        dis_word = []
        dis_word.append(min_edit_dist(miss_word,wordlist[i]))
        dis_word.append(wordlist[i])
        dis.append(dis_word)
    return dis

In [ ]:
wordlist = wn.words()
f_list,sak1,sak5,sak10 = [],[],[],[]
count = 0;
try1 = {}
for w in misspelled:
    m_c = w[0]
    for w1 in wordlist:
        if(len(w1)>=len(w)-2 and len(w1)<=(len(w)+2)):
            c = w1[0]
            if(m_c == c):
                f_list.append(w1)
    q = sim_word(f_list,w)
    df = pd.DataFrame(q, columns=["MED",w])
    df = df.sort_values("MED", ignore_index=True)
    p = int(min_edit_dist(w,spelled[count]))
    qrel = {w : {spelled[count]:10}}
    run = {w : {
        df[w][9]:1,
        df[w][8]:2,
        df[w][7]:3,
        df[w][6]:4,
        df[w][5]:5,
        df[w][4]:6,
        df[w][3]:7,
        df[w][2]:8,
        df[w][1]:9,
        df[w][0]:10,
    }}
    eva = pytrec_eval.RelevanceEvaluator(qrel,{'map','ndcg'})
    pytrec_eval.supported_measures
    print("\nS@k for top 10 : ",json.dumps(eva.evaluate(run), indent=1))
    
    print((df[w][0],spelled[count]))
    print((df[w][4],df[w][9]))
    
    count = count+1
    if(count == 20): break
    '''for i in sorted(q.keys()):
        print((i,q[i]),end=",")
    print(w)
    print("end")'''